In [1]:
import dgl
from dgl.data import DGLDataset
import torch
import os
import pandas as pd

Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


DGL backend not selected or invalid.  Assuming PyTorch for now.


In [102]:
class COMP4222Dataset(DGLDataset):
    def __init__(self):
        super().__init__(name='comp-4222')

    def process(self):
        df_startups = pd.read_csv('startups_formatted.csv')
        df_investors = pd.read_csv('investors_formatted.csv')
        df_investments = pd.read_csv('funding_round_formatted.csv')

        data_dict = {
            ("investor", "raise", "startup"): (torch.tensor(df_investments.investor_object_id.values.tolist()), torch.tensor(df_investments.funded_object_id.values.tolist()))
            }     
        self.graph = dgl.heterograph(data_dict)
        
        edge_feature = [i for i in df_investments.columns if i not in ["funding_round_id", "funded_object_id", "investor_object_id"]]

        graph.nodes['investor'].data['features'] = torch.tensor(df_investors.iloc[:, 2:].to_numpy())
        graph.nodes['startup'].data['features'] = torch.tensor(df_startups.iloc[:, 2:].to_numpy())

        graph.edges['raise'].data['features'] = torch.tensor(df_investments[edge_feature].to_numpy())

    def __getitem__(self, i):
        return self.graph

    def __len__(self):
        return 1

dataset = COMP4222Dataset()
graph = dataset[0]

print(graph)

Graph(num_nodes={'investor': 7594, 'startup': 21485},
      num_edges={('investor', 'raise', 'startup'): 60983},
      metagraph=[('investor', 'startup', 'raise')])
